# RQ1 mutation map

## To what extent the sota benchmark is mutable according to wasm-mutate?


In [56]:
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import json

def load_sets(name):
    a = open(name, 'r').read()
    data = json.loads(a)
    
    sets = {}
    
    upto=-1
    for i, t in enumerate(data[:upto]):
        name = t['id']
        mutations = t['mutations']
        
        for m in mutations:
            if m['class_name'] not in sets:
                sets[m['class_name']] = set()
            sets[m['class_name']].add(i)
            
    for k, v in sets.items():
        print(k, len(v))
    return list(zip(sets.keys(), sets.values())), len(data[:upto])

def get_node_sets(nodes, total, operator="+"):
        
        
    def merge(sets):
        #print(sets)
        r = sets[0]
        
        if operator == "+":
            op = r.union
        elif operator == "i":
            op = r.intersection
        
        for c in sets[1:]:
            r = op(c)
            
            if operator == "+":
                op = r.union
            elif operator == "i":
                op = r.intersection
        
        return r
    
    def get_combinations(sets, size=2, plot_percentage=True):
        #names, sets = [s[0] for s in sets], [s[1] for s in sets]
        combs = [c for c in itertools.combinations(sets, size)]
        
        r = []
        
        edges = []
        for c in combs:
            #print("Combination", c)
            names, sets = [ cc[0] for cc in c ], [cc[1] for cc in c]
            merged = merge(sets)
            
            #print(names)
            id = f"-{operator}-".join(names)
            s = (len(merged)/total) if plot_percentage else len(merged)
            #id = f"{id}({s})"
            print(id)
            
            for i in sorted(names):
                if id != f"{i}":
                    
                    color='black'

                    if len(merged) == 0:
                        color='C1'
                    edges.append((id, f"{i}", dict(color='black')))
                    
            color='C0'
            
            if size==1:
                color='C2'
            elif len(merged) == 0:
                color='C1'
            #print(len(merged), s)
            label = id.replace(f"-{operator}-", "\n")
                
            r.append((id, dict(count=len(merged), size=len(merged), set=merged, id=id, label=label, s=s, color=color)))
            
        return r, edges
    
    allnodes = []
    alledges = []
    for k in range(1, len(nodes) + 1):
        newsets, edges = get_combinations(nodes, k)
        
        allnodes +=  newsets
        alledges += edges
        
    return allnodes, alledges

    
sets, total = load_sets("all.json")
print(total)

nodes, edges = get_node_sets(sets, total, operator='i')

print(len(nodes), len(edges))


    
scale_factor = 10
node_sizes = [ scale_factor+n[1]['size']*scale_factor for n in nodes ]
colors = [ n[1]['color'] for n in nodes ]
edge_colors = [e[2]['color'] for e in edges] 


G = nx.Graph()
G.add_nodes_from(nodes)

node_labels = { n[1]['id'] : "%s\n%s"%(n[1]['label'], n[1]['s']) for n in nodes }

#for e in edges:
    #print(e)
G.add_edges_from(edges)
    
nx.draw(G,with_labels=True,
         pos=nx.shell_layout(G),
         node_color = colors,
         node_size=node_sizes,
         labels=node_labels
         #edge_color=edge_colors
        )

plt.rcParams["figure.figsize"] = (1200, 1200, dpi=50)
plt.axis('off')
plt.savefig("plot.png")

SyntaxError: invalid syntax (3963467581.py, line 131)

In [1]:
r=set([1,2])
print(r.intersection(r))

{1, 2}
